In [10]:
import boto3
import os
import pathlib
s = boto3.session.Session(profile_name='sanlam-dev-admin')

ssm = s.client('ssm')
test = ssm.list_command_invocations(InstanceId="i-0cd7c684127cf040d")['CommandInvocations'][0]
print(test)

{'CommandId': '8858afcb-f950-4e98-9c56-f935731ffd38', 'InstanceId': 'i-0cd7c684127cf040d', 'InstanceName': 'ip-10-0-1-239.eu-west-1.compute.internal', 'Comment': '', 'DocumentName': 'MWR-SSM-Test-Case', 'DocumentVersion': '$DEFAULT', 'RequestedDateTime': datetime.datetime(2022, 9, 22, 11, 9, 36, 945000, tzinfo=tzlocal()), 'Status': 'Success', 'StatusDetails': 'Success', 'StandardOutputUrl': '', 'StandardErrorUrl': '', 'CommandPlugins': [], 'ServiceRole': '', 'NotificationConfig': {'NotificationArn': '', 'NotificationEvents': [], 'NotificationType': ''}, 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '', 'CloudWatchOutputEnabled': False}}


In [ ]:
import boto3
import argparse
import json
import logging
import pathlib
from pprint import pprint
s = boto3.session.Session(profile_name='sanlam-dev-admin')


def save_data(filename,data):
    # Make output dir if it doesnt exist
    output_dir = pathlib.Path(__file__).parent.absolute() / 'output'
    output_dir.mkdir(exist_ok=True)
    filepath = output_dir / filename
    with filepath.open("w") as write_file:
        json.dump(data, write_file, indent=4, default=str)


def check_ecs_exec_enabled(specific_region):
    if specific_region == None:
        regions = s.get_available_regions('ecr')
        regions = [i for i in regions if i not in not_allowed_regions]
    else:
        regions = [specific_region]
    
    repository_access_policies = []
    for region in regions:
        ecr = s.client('ecr',region_name=region)
        respositories = ecr. .list_clusters()['clusterArns']
        for cluster in clusters:
            tasks = ecs.list_tasks(cluster=cluster)['taskArns']
            if not tasks:
                continue
            # for task in tasks:
            task_data = ecs.describe_tasks(tasks=tasks,cluster=cluster)['tasks']
            for individual_task_data in task_data:
                if "enableExecuteCommand" in individual_task_data.keys():
                    print(f"Looks like ECS exec is enabled for cluster {cluster}")
                for container_definition in individual_task_data['containers']:
                    if not "managedAgents" in container_definition.keys():
                        continue
                    for managed_agent in container_definition['managedAgents']:
                        if managed_agent["name"] == "ExecuteCommandAgent":
                            print("Found an ECS container with the ECS Exec Managed Agent on it")
                            print(task_data)
                            continue    
            print(f"Nothing found for {cluster}")
        total_task_data.append(task_data)

    save_data(f"{account_id}-ecs-task_data.json",total_task_data)

s = boto3.session.Session(profile_name='sanlam-dev-admin')

# Check that the profile creds are valid
sts = s.client('sts')
try:
    sts_info = sts.get_caller_identity()
    account_id = sts_info.get('Account')
except sts.exceptions.ClientError as e:
    print(f"{e}")
    print(f"Exiting..")
    exit()

not_allowed_regions = []

check_ecs_exec_enabled('eu-west-1')
# ecs.execute_command(
#     command='cat /etc/passwd',
#     con

# )
# print(clusters)
# # services = ecs.list_services()
# tasks = ecs.list_tasks()

SyntaxError: invalid syntax (2809459617.py, line 29)